In [ ]:
from datasets import load_dataset, DownloadConfig
import os, shutil, random, io
from PIL import Image
from tqdm import tqdm
from huggingface_hub import notebook_login

In [1]:
# download_20k.py  ── 抓 20 000 張 ImageNet，已存在就跳過
from datasets import load_dataset
from PIL import Image
from tqdm import tqdm
import pathlib, os, io, random, hashlib

DATASET = "jovianzm/Pexels-400k"
SPLIT   = "train"
TARGET  = 50_000                       # 最終要湊滿的張數
DEST    = pathlib.Path("data/raw/pexels")
DEST.mkdir(parents=True, exist_ok=True)

# ① 先統計已經有多少張
exist = {p.name for p in DEST.glob("*.jpg")}
done  = len(exist)
print(f"已存在 {done} 張，目標 {TARGET}")

if done >= TARGET:
    print("✓ 目標已滿，無需下載"); exit()

# ② streaming + shuffle
ds = load_dataset(DATASET, split=SPLIT, streaming=True)
stream = ds.shuffle(seed=42, buffer_size=20_000)

# ③ 邊串流邊存檔，檔名重複就 continue
saved = 0
for ex in tqdm(stream, unit="img"):
    if done + saved >= TARGET:
        break

    img = ex["image"]
    if isinstance(img, bytes):
        img = Image.open(io.BytesIO(img))
    img = img.convert("RGB")

    # 檔名：<label>_<序號>.jpg ；重複檢查只看檔名（速度最快）
    fname = f"{ex['label']:04d}_{done+saved:08d}.jpg"
    if fname in exist:
        continue

    img.save(DEST / fname, quality=95)
    exist.add(fname)
    saved += 1

print(f"✅ 下載完成，共 {done+saved}/{TARGET} 張位於 {DEST}"# ╔═╗  Download lehduong/flux_generated  ═══════════════════════════════╗


SyntaxError: incomplete input (2667755865.py, line 45)

In [2]:
# 參數：自行修改
DATASET   = "jovianzm/Pexels-400k"   # HF 資料集名稱
SPLIT     = "train"                     # 試過若無 train 可改 "default"
TARGET    = 50_000                      # 想要抓的總張數
DEST      = "data/raw/fake/pexels"
BUFFER    = 12_000                      # shuffle 緩衝
SEED      = 42
USE_HASH  = False                       # 遇到相同檔名就跳過；True ➜ 用 SHA-1 去重

# ──────────────────────────────────────────────────────────────────────
import pathlib, os, io, hashlib, warnings
from datasets import load_dataset
from PIL import Image
from tqdm import tqdm

warnings.filterwarnings("ignore", category=UserWarning, module="PIL.")

dest = pathlib.Path(DEST)
dest.mkdir(parents=True, exist_ok=True)

# 1️⃣ 先統計已存檔
exist = {p.name for p in dest.glob("*.jpg")}
done  = len(exist)
if done >= TARGET:
    print(f"✓ 目前 {done}/{TARGET}，已達標") ; raise SystemExit

print(f"已有 {done} 張，開始補到 {TARGET}…")

# 2️⃣ 取得 streaming Dataset
ds = load_dataset(DATASET, split=SPLIT, streaming=True)
stream = ds.shuffle(seed=SEED, buffer_size=BUFFER)

def make_key(img, label, idx):
    if USE_HASH:
        h = hashlib.sha1(img.tobytes()).hexdigest()
        return f"{h}.jpg"
    else:
        return f"{label:04d}_{idx:08d}.jpg"

saved = 0
for ex in tqdm(stream, unit="img"):
    if done + saved >= TARGET:
        break

    # 讀到 PIL.Image
    img = ex["image"]
    if isinstance(img, bytes):
        img = Image.open(io.BytesIO(img))
    img = img.convert("RGB")

    key = make_key(img, ex.get("label", 0), done + saved)
    if key in exist:
        continue

    img.save(dest / key, quality=95)
    exist.add(key)
    saved += 1

print(f"✅ 新增 {saved} 張，總計 {len(exist)}/{TARGET}")
# ╚════════════════════════════════════════════════════════════════════╝



/home/yaya/miniforge3/envs/imgfeat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


已有 0 張，開始補到 50000…


0img [00:03, ?img/s]


KeyError: 'image'

In [1]:
# ╔═ Notebook-friendly Downloader ═╗
DATASET = "ProGamerGov/synthetic-dataset-1m-dalle3-high-quality-captions"
SPLIT   = "train"
TARGET  = 50_000
DEST    = "/home/yaya/aigen/data/raw/fake/dalle3"   # ← 用絕對路徑
BUFFER  = 12_000
SEED    = 42
USE_HASH= False

import pathlib, os, io, hashlib, warnings, itertools
from datasets import load_dataset
from PIL import Image
from tqdm.notebook import tqdm            # ✅

warnings.filterwarnings("ignore", category=UserWarning, module="PIL.")

dest = pathlib.Path(DEST)
dest.mkdir(parents=True, exist_ok=True)

exist = {p.name for p in dest.glob("*.jpg")}
done  = len(exist)
print(f"已有 {done} 張，目標 {TARGET}")

if done >= TARGET:
    print("✓ 已達標！"); raise SystemExit

ds      = load_dataset(DATASET, split=SPLIT, streaming=True)
stream  = ds.shuffle(seed=SEED, buffer_size=BUFFER)

def make_key(img, label, idx):
    return (hashlib.sha1(img.tobytes()).hexdigest() if USE_HASH
            else f"{label:04d}_{idx:08d}") + ".jpg"

saved = 0
for ex in tqdm(stream, unit="img", total=TARGET-done):
    if done + saved >= TARGET:
        break
    img = ex["image"]
    if isinstance(img, bytes):
        img = Image.open(io.BytesIO(img))
    img = img.convert("RGB")

    key = make_key(img, ex.get("label",0), done+saved)
    if key in exist:           # 跳重複
        continue

    img.save(dest / key, quality=95)
    exist.add(key); saved += 1

print(f"✅ 新增 {saved} 張，總計 {len(exist)}/{TARGET}")
# ╚═══════════════════════════════╝


已有 0 張，目標 50000


Resolving data files:   0%|          | 0/69 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/69 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?img/s]

KeyboardInterrupt: 

In [ ]:
#ProGamerGov/synthetic-dataset-1m-dalle3-high-quality-captions

In [ ]:
# ╔═ Notebook-friendly Downloader (v2) ═╗
DATASET   = "OpenDatasets/dalle-3-dataset"
SPLIT     = "train"
TARGET    = 50_000
DEST      = "/home/yaya/aigen/data/raw/fake/dalle3"
BUFFER    = 12_000
SEED      = 42
USE_HASH  = False          # True → 用 SHA-1 去重
SAVE_CAP  = False          # True → 同名 .txt 存 caption
FLUSH_EVERY = 1_000        # 每抓 N 張 flush 一次

# ──────────────────────────────────────
import pathlib, os, io, hashlib, warnings, time
from datasets import load_dataset
from PIL import Image
from tqdm.auto import tqdm

warnings.filterwarnings("ignore", category=UserWarning, module="PIL.")
dest = pathlib.Path(DEST); dest.mkdir(parents=True, exist_ok=True)

# 1️⃣ 讀續跑紀錄
done_file = dest / "done.txt"
if done_file.exists():
    with done_file.open() as f:
        exist = set(line.strip() for line in f)
else:
    exist = set()

done = len(exist)
print(f"已有 {done} 張，目標 {TARGET}")
if done >= TARGET:
    print("✓ 已達標！"); raise SystemExit

# 2️⃣ 建立 streaming dataset
ds = load_dataset(DATASET, split=SPLIT, streaming=True)
stream = ds.shuffle(seed=SEED, buffer_size=BUFFER)

def make_key(img, label, idx):
    if USE_HASH:
        h = hashlib.sha1(img.tobytes()).hexdigest()
        return f"{h}.jpg"
    else:
        return f"{label:04d}_{idx:08d}.jpg"

saved = 0; tic = time.time()
bar = tqdm(total=TARGET-done, unit="img")
out_f = done_file.open("a")

for ex in stream:
    if done + saved >= TARGET:
        break

    img = ex["image"]
    if isinstance(img, bytes):
        img = Image.open(io.BytesIO(img))
    img = img.convert("RGB")

    key = make_key(img, ex.get("label", 0), done + saved)
    if key in exist:
        continue

    # 3️⃣ 儲存圖片 (+ caption)
    img.save(dest / key, quality=95)
    if SAVE_CAP:
        cap_path = dest / f"{key[:-4]}.txt"
        cap_path.write_text(ex.get("text", ""), encoding="utf-8")

    exist.add(key); out_f.write(key + "\n"); saved += 1
    bar.update(1)

    # 4️⃣ 每 N 張 flush + 顯示速率
    if saved % FLUSH_EVERY == 0:
        out_f.flush(); os.fsync(out_f.fileno())
        speed = saved / (time.time() - tic)
        bar.set_postfix_str(f"{speed:.1f} img/s")

out_f.close(); bar.close()
print(f"✅ 新增 {saved} 張，總計 {len(exist)}/{TARGET}")
# ╚════════════════════════════════════╝


已有 0 張，目標 50000


Resolving data files:   0%|          | 0/67 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?img/s]

In [5]:
# ╔═╗  batch-zip-to-10k ─ 每 1萬 張圖片一包 ══════════════════════════════╗
# 設定區 ── 改成自己的路徑 / 參數
SRC_DIR  = "data/raw/fake/midjourney-v6-llava"   # 放圖片的資料夾（所有圖平放）
DST_DIR  = "data/zips/midjourney-v6-llava"               # 壓縮檔要存哪
CHUNK    = 10_000                    # 每 ZIP 裡放多少張
EXTS     = [".jpg", ".jpeg", ".png", ".webp", ".bmp"]  # 要抓的副檔名

# ──────────────────────────────────────────────────────────────────────
import pathlib, zipfile, tqdm, os

src = pathlib.Path(SRC_DIR)
dst = pathlib.Path(DST_DIR); dst.mkdir(parents=True, exist_ok=True)

imgs = sorted([p for p in src.iterdir() if p.suffix.lower() in EXTS])
total = len(imgs)
print(f"找到 {total:,} 張影像，開始打包…")

batches = [imgs[i:i+CHUNK] for i in range(0, total, CHUNK)]

for idx, batch in enumerate(batches):
    zip_path = dst / f"img_{idx:04d}.zip"
    if zip_path.exists():
        print(f"✓ {zip_path.name} 已存在，跳過")
        continue

    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for p in tqdm.tqdm(batch, desc=zip_path.name, unit="img"):
            zf.write(p, arcname=p.name)   # 只存檔名，不帶資料夾層
    print(f"✅ {zip_path.name} 完成（{len(batch)} 張）")

print("🎉 全部批次壓縮完成！")
# ╚════════════════════════════════════════════════════════════════════╝


找到 50,000 張影像，開始打包…


img_0000.zip: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:33<00:00, 107.25img/s]


✅ img_0000.zip 完成（10000 張）


img_0001.zip: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:33<00:00, 106.45img/s]


✅ img_0001.zip 完成（10000 張）


...

✅ img_0002.zip 完成（10000 張）


img_0003.zip: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:33<00:00, 106.79img/s]


✅ img_0003.zip 完成（10000 張）


img_0004.zip: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:34<00:00, 105.47img/s]

✅ img_0004.zip 完成（10000 張）
🎉 全部批次壓縮完成！


rafaelpadilla/coco2017
ProGamerGov/synthetic-dataset-1m-dalle3-high-quality-captions
brivangl/midjourney-v6-llava

In [1]:
# cell 1 ── 載入資料表
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/home/yaya/ai-detect-proj/unsplash-research-dataset-lite-latest").expanduser()

# Lite 版的表被切成 photos.csv000、photos.csv001 ...
photo_parts = sorted(DATA_DIR.glob("photos.csv*"))
df_photos = pd.concat(
    [pd.read_csv(p, sep='\t', compression='infer', low_memory=False)  # 檔案實際是 TSV
     for p in photo_parts],
    ignore_index=True
)

print("共有", len(df_photos), "張照片的紀錄")
df_photos.head()


共有 25002 張照片的紀錄


,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash,[ZoneTransfer]
0,oSf8ePoG9NU,https://unsplash.com/photos/oSf8ePoG9NU,https://images.unsplash.com/20/frozen-grass.JPG,2013-12-04 14:58:49,t,4000.0,2667.0,1.5,NaN,andrekoch,...,NaN,4296224.0,30072.0,black road in between white and brown grass ac...,NaN,NaN,NaN,NaN,LPBM;~RkM|of~VRkNHof%1RjRkof,NaN
1,DlsOa5moK4w,https://unsplash.com/photos/DlsOa5moK4w,https://images.unsplash.com/reserve/dRA4UuMBR2...,2014-07-10 17:19:38,t,5184.0,3456.0,1.5,Fresh blueberries,majkmmiklavc,...,NaN,8731049.0,26411.0,selective focus of blueberry plant,NaN,NaN,NaN,NaN,LiJ+M4s=-pj;~qx]RpbXx1ogRkob,NaN
2,XBGacbT3vXI,https://unsplash.com/photos/XBGacbT3vXI,https://images.unsplash.com/photo-143633523196...,2015-07-08 05:54:38.486126,t,6000.0,4000.0,1.5,NaN,shooshanig,...,NaN,2224540.0,19250.0,photography of calm beach side,NaN,NaN,NaN,NaN,LSL}HBR*a{fP_NWVoej[x^s.j[ay,NaN
3,FjikPptEbZg,https://unsplash.com/photos/FjikPptEbZg,https://images.unsplash.com/flagged/photo-1441...,2015-08-31 07:12:27.700568,f,3008.0,2008.0,1.5,NaN,websanya,...,Saint-Étienne-de-Tinée,45536.0,421.0,green trees on mountain under white clouds dur...,NaN,NaN,NaN,NaN,LyF7Cux[M{kB?^t6azbHRjjrt7ae,NaN
4,PXdBkNF8rlk,https://unsplash.com/photos/PXdBkNF8rlk,https://images.unsplash.com/photo-144596683821...,2015-10-27 17:30:25.724092,t,3000.0,2000.0,1.5,Icebergs of Iceland’s Vatnajokull,andersjilden,...,NaN,7394191.0,43442.0,icebergs floating on body of water during daytime,NaN,NaN,NaN,NaN,LQG]];?b~qayBtIU9Ft79xj[tRM|,NaN


In [2]:
# cell 2 ── 批次下載
import requests, shutil, time, tqdm, os

SAVE_DIR = Path("/home/yaya/ai-detect-proj/data/raw/real/unsplash")
SAVE_DIR.mkdir(exist_ok=True)

# 建議用 raw_url；若想配合 Unsplash 的下載統計，改用 download_location
subset = df_photos.sample(25000, random_state=42)   # 先隨機抓 200 張試跑

for _, row in tqdm.tqdm(subset.iterrows(), total=len(subset)):
    url  = row['photo_image_url']            # 或 row['download_location']
    pid  = row['photo_id']
    ext  = ".jpg"                    # Unsplash 皆為 JPEG
    dest = SAVE_DIR / f"{pid}{ext}"
    if dest.exists(): 
        continue                     # 已下載過就略過
    try:
        with requests.get(url, stream=True, timeout=30) as r:
            r.raise_for_status()
            with open(dest, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
        time.sleep(0.3)              # 避免對伺服器造成過大負載
    except Exception as e:
        print("✗", pid, e)


 45%|████▍     | 11219/25000 [00:00<00:00, 38962.00it/s]

✗ rsJtMXn3p_c HTTPSConnectionPool(host='images.unsplash.com-grass-sun.jpg', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x72683e22ba50>: Failed to resolve 'images.unsplash.com-grass-sun.jpg' ([Errno -2] Name or service not known)"))


 60%|██████    | 15124/25000 [00:00<00:00, 37169.98it/s]

✗ vigsqYux_-8 HTTPSConnectionPool(host='images.unsplash.com_thebeach.jpg', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x72683c8be5d0>: Failed to resolve 'images.unsplash.com_thebeach.jpg' ([Errno -2] Name or service not known)"))


 66%|██████▌   | 16381/25000 [10:38<1:18:55,  1.82it/s] 

...

 88%|████████▊ | 22006/25000 [1:28:30<34:24,  1.45it/s]  

✗ nan Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


 90%|█████████ | 22565/25000 [1:36:02<40:12,  1.01it/s]  

✗ nan Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


100%|██████████| 25000/25000 [2:09:04<00:00,  3.23it/s]  


In [1]:
from datasets import load_dataset

# 讀取 train split（只有 train 這一個 split）
ds = load_dataset(
    "wtcherr/unsplash_20k",
    split="train",
    streaming=True,     # 若想邊用邊載可改成 True
    trust_remote_code=True
)

print("共有", len(ds), "筆")       # 約 19000
print(ds.column_names)            # ['caption', 'image', 'link', ...]


/home/yaya/miniforge3/envs/imgfeat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'wtcherr/unsplash_20k' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


TypeError: object of type 'IterableDataset' has no len()

In [4]:
import os
from pathlib import Path
from tqdm import tqdm

# 1. 目的資料夾
out_dir = Path("/home/yaya/ai-detect-proj/data/raw/fake/dalle3_png")
out_dir.mkdir(parents=True, exist_ok=True)

# 2. 下載與儲存（最多 20 000 張）
MAX_IMAGES = 20000

for idx, row in enumerate(tqdm(ds.take(MAX_IMAGES), desc="save png")):
    img = row["image"]        # PIL.Image 物件
    img.save(
        out_dir / f"{idx:06d}.png",
        format="PNG",
        compress_level=6       # 0~9，數字越大壓縮越高（預設 6）
    )


save png: 837it [08:27,  1.65it/s]


KeyboardInterrupt: 

In [ ]:
from datasets import load_dataset
import requests, os, time, random, threading
from pathlib import Path
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed, wait, FIRST_COMPLETED

# ========= 參數 =========
DATASET_ID    = "bitmind/open-images-v7"
SPLIT         = "train"                 # "train" | "validation" | "test"
TARGET_GOOD   = 50000                   # 你想要的「合格」張數（進度條看這個）
OUT_DIR       = Path("/home/yaya/ai-detect-proj/data/raw/real/open-images-v7")
SEED          = 72
SHUF_BUFFER   = 200_000
TIMEOUT       = 25
RETRIES       = 3
SLEEP_BASE    = 1.0
VERIFY_IMAGE  = True
MIN_SIDE      = 512                     # 小於此值視為不合格（skip_small）
SAVE_AS_PNG   = False
MAX_WORKERS   = 32                      # 併發
MAX_INFLIGHT  = MAX_WORKERS * 4         # 同時在跑/排隊的任務上限，減少超量下載

OUT_DIR.mkdir(parents=True, exist_ok=True)
random.seed(SEED)

# ========= 工具 =========
def _ext_from_url(u: str) -> str:
    ext = os.path.splitext(urlparse(u).path)[1].lower()
    return ext if ext in [".jpg", ".jpeg", ".png", ".webp", ".bmp"] else ".jpg"

def _build_path(idx: int, url: str) -> Path:
    return OUT_DIR / (f"{idx:06d}.png" if SAVE_AS_PNG else f"{idx:06d}{_ext_from_url(url)}")

def _save_bytes_as_img(data: bytes, path: Path):
    tmp = path.with_suffix(path.suffix + ".part")
    if SAVE_AS_PNG:
        img = Image.open(BytesIO(data)).convert("RGB")
        img.save(tmp, format="PNG", compress_level=6)
        os.replace(tmp, path)
    else:
        with open(tmp, "wb") as f:
            f.write(data)
        os.replace(tmp, path)

_thread_local = threading.local()
def _get_session() -> requests.Session:
    s = getattr(_thread_local, "session", None)
    if s is None:
        s = requests.Session()
        s.headers.update({"User-Agent": "Mozilla/5.0 (compatible; open-images-downloader)"})
        _thread_local.session = s
    return s

def _download_one(idx: int, url: str):
    """回傳: ('ok' | 'skip_exists' | 'skip_small' | 'bad_verify' | 'bad_fail', msg)"""
    if not url:
        return ("bad_fail", "no_url")

    out_path = _build_path(idx, url)
    if out_path.exists():
        return ("skip_exists", "exists")

    backoff = SLEEP_BASE
    for attempt in range(1, RETRIES + 1):
        try:
            r = _get_session().get(url, timeout=TIMEOUT)
            if r.status_code != 200:
                raise RuntimeError(f"http {r.status_code}")
            data = r.content

            if VERIFY_IMAGE or MIN_SIDE > 0 or SAVE_AS_PNG:
                try:
                    img = Image.open(BytesIO(data))
                    img.verify()
                    img = Image.open(BytesIO(data))
                    if MIN_SIDE > 0:
                        w, h = img.size
                        if min(w, h) < MIN_SIDE:
                            return ("skip_small", f"{w}x{h}")
                except Exception as e:
                    return ("bad_verify", str(e))

            _save_bytes_as_img(data, out_path)
            return ("ok", "saved")
        except Exception as e:
            if attempt >= RETRIES:
                return ("bad_fail", str(e))
            time.sleep(backoff)
            backoff *= 2.0

# ========= 準備資料（串流 + 打散）=========
ds = load_dataset(DATASET_ID, split=SPLIT, streaming=True)
ds = ds.shuffle(seed=SEED, buffer_size=SHUF_BUFFER)
it = iter(ds)

# 取資料時的索引（用來命名檔案）
next_idx = 0

def _next_task():
    """讀取下一筆 (idx, url)，若無法再取回傳 None"""
    global next_idx
    while True:
        try:
            row = next(it)   # 可能 StopIteration
        except StopIteration:
            return None
        url = row.get("url") or row.get("URL")
        # 先把已存在的檔案過濾掉，不丟進執行緒池（不影響進度條 total）
        path = _build_path(next_idx, url if url else "")
        if url and not path.exists():
            idx = next_idx
            next_idx += 1
            return (idx, url)
        else:
            next_idx += 1
            # 單純略過已存在／無 URL 的案例，繼續抓下一筆

# ========= 併發下載（進度條只計「ok」）=========
ok = skip_exists = skip_small = bad_verify = bad_fail = 0

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = set()

    # 先灌一批任務
    while len(futures) < MAX_INFLIGHT:
        task = _next_task()
        if task is None:
            break
        futures.add(ex.submit(_download_one, *task))

    with tqdm(total=TARGET_GOOD, desc=f"ok-only ({SPLIT})") as pbar:
        while futures and ok < TARGET_GOOD:
            done, futures = wait(futures, return_when=FIRST_COMPLETED)
            for fut in done:
                status, _ = fut.result()
                if status == "ok":
                    ok += 1
                    pbar.update(1)                 # ★ 只在 ok 時更新進度條
                elif status == "skip_exists":
                    skip_exists += 1
                elif status == "skip_small":
                    skip_small += 1
                elif status == "bad_verify":
                    bad_verify += 1
                else:
                    bad_fail += 1

                # 補進新的任務，維持 in-flight 數量
                if ok < TARGET_GOOD:
                    task = _next_task()
                    if task is not None:
                        futures.add(ex.submit(_download_one, *task))

        # 如果還有未完成但我們已經達標，就嘗試取消
        if ok >= TARGET_GOOD:
            ex.shutdown(wait=False, cancel_futures=True)

print(f"Done. ok={ok}, skip_exists={skip_exists}, skip_small={skip_small}, "
      f"bad_verify={bad_verify}, bad_fail={bad_fail}")
print("Saved to:", OUT_DIR)


/home/yaya/miniforge3/envs/imgfeat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ok-only (train):   0%|          | 79/50000 [53:49<503:04:41, 36.28s/it]  